# Extracción de Issues Detallados desde SonarCloud

Este notebook está diseñado para extraer todos los issues (problemas) detallados de los proyectos de SonarCloud de los estudiantes, proporcionando un análisis profundo de la calidad del código a nivel de issue individual.

## Objetivos:
1. **Cargar datos** de proyectos de estudiantes desde el CSV con columnas Sonar_Ap1 y Sonar_Ap2
2. **Extraer issues detallados** desde SonarCloud API para cada proyecto utilizando `/api/issues/search`
3. **Procesar y categorizar** issues por severidad, tipo y regla aplicada
4. **Generar análisis** de patrones de calidad de código por estudiante y assignment
5. **Exportar resultados** para análisis estadísticos posteriores

## Información Extraída por Issue:
- **Identificación**: Issue key, project key, regla aplicada
- **Clasificación**: Severidad (CRITICAL, MAJOR, MINOR, etc.), tipo (BUG, VULNERABILITY, CODE_SMELL)
- **Localización**: Archivo/componente afectado, línea de código específica
- **Descripción**: Mensaje detallado del problema detectado
- **Contexto**: Información del estudiante y assignment (AP1/AP2)

## Diferencia con el Notebook Anterior:
- **Enfoque**: Issues individuales vs. métricas agregadas
- **Granularidad**: Nivel de línea de código vs. nivel de proyecto
- **Análisis**: Patrones específicos de problemas vs. estadísticas generales
- **Volumen**: Mayor cantidad de datos detallados por proyecto

In [ ]:
# Instalación de dependencias (ejecutar solo si es necesario)
import subprocess
import sys

def install_package(package):
    """Instalar paquete si no está disponible"""
    try:
        __import__(package)
        print(f"✅ {package} ya está instalado")
    except ImportError:
        print(f"🔄 Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado exitosamente")

# Lista de paquetes requeridos
required_packages = ['pandas', 'requests']

print("🔧 Verificando e instalando dependencias...")
for package in required_packages:
    install_package(package)

print("✅ Todas las dependencias están listas")

In [ ]:
# Importar librerías necesarias
import pandas as pd
import requests
import json
import base64
import time
from datetime import datetime
from typing import Dict, List, Optional
import warnings
import logging
from functools import wraps
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")
print("📝 Pandas version:", pd.__version__)
print("🌐 Requests disponible para API calls")
print("🕐 Datetime y logging configurados")

In [ ]:
# Funciones utilitarias para optimizar el código
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def timer_decorator(func):
    """Decorador para medir tiempo de ejecución de funciones"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"⏱️  {func.__name__} ejecutado en {end_time - start_time:.2f} segundos")
        return result
    return wrapper

def clean_message_for_csv(message):
    """Limpiar mensaje para formato CSV"""
    if not message:
        return ""
    # Escapar comillas dobles duplicándolas y envolver en comillas
    cleaned = message.replace('"', '""')
    return f'"{cleaned}"'

def safe_divide(numerator, denominator):
    """División segura que evita división por cero"""
    return numerator / denominator if denominator != 0 else 0

def format_percentage(value, total):
    """Formatear porcentaje de manera consistente"""
    return f"{safe_divide(value, total) * 100:.1f}%"

def print_section_header(title, char="=", width=50):
    """Imprimir headers de sección consistentes"""
    print(f"\n{title}")
    print(char * width)

print("🛠️  Funciones utilitarias cargadas")
print("📊 Timer, limpieza de datos y formateo disponibles")

## 1. Configuración de la API de SonarCloud

Configuramos los parámetros necesarios para conectarnos a SonarCloud y extraer issues detallados:

- **Token de autenticación**: Credenciales para acceder a la API de SonarCloud
- **Headers de autenticación**: Configuración básica con base64 encoding
- **Organización**: Clave de la organización en SonarCloud (`tesisenel`)
- **Endpoints**: URLs para búsqueda de proyectos e issues

### Diferencias con Extracción de Métricas:
- **Endpoint principal**: `/api/issues/search` (vs `/api/measures/component`)
- **Paginación**: Manejo de grandes volúmenes de issues por proyecto
- **Filtros**: Capacidad de filtrar por tipo de issue, severidad, estado
- **Detalle**: Información específica de ubicación (archivo, línea)

In [ ]:
# Configuración de SonarCloud API para extracción de issues
from functools import lru_cache

# Configuración de autenticación (usando el mismo token del notebook anterior)
SONAR_TOKEN = "cc64d7ea652e603cacbc87bbb9c7b550efee7353"
SONAR_BASE_URL = "https://sonarcloud.io/api"
SONAR_ORGANIZATION = "tesisenel"

# Cache para headers de autenticación
@lru_cache(maxsize=1)
def get_auth_headers():
    """Crear headers de autenticación para SonarCloud API (con caché)"""
    auth_string = f"{SONAR_TOKEN}:"
    auth_bytes = auth_string.encode('ascii')
    auth_b64 = base64.b64encode(auth_bytes).decode('ascii')
    
    return {
        'Authorization': f'Basic {auth_b64}',
        'Content-Type': 'application/json'
    }

# Configuración de parámetros de extracción
ISSUES_CONFIG = {
    'page_size': 500,  # Máximo permitido por SonarCloud
    'max_retries': 3,
    'retry_delay': 5.0,
    'batch_delay': 2.0,
    'timeout': 30
}

# Tipos de issues que vamos a extraer
ISSUE_TYPES = ['BUG', 'VULNERABILITY', 'CODE_SMELL']
SEVERITIES = ['BLOCKER', 'CRITICAL', 'MAJOR', 'MINOR', 'INFO']

print("🔐 Configuración de autenticación preparada")
print(f"🏢 Organización: {SONAR_ORGANIZATION}")
print(f"📊 Page size: {ISSUES_CONFIG['page_size']} issues por página")
print(f"🔄 Reintentos: {ISSUES_CONFIG['max_retries']} máximo")
print("✅ Headers de API configurados con caché")
print("⚠️  Token expuesto temporalmente para pruebas")

## 2. Carga de Datos de Estudiantes

Cargamos el mismo archivo CSV utilizado en el notebook anterior, que contiene la información de los estudiantes incluyendo las columnas `Sonar_Ap1` y `Sonar_Ap2` con las claves de los proyectos en SonarCloud.

### Reutilización de Funciones:
Utilizaremos la misma función `extract_project_keys()` para extraer las claves de proyecto, manteniendo consistencia con el análisis de métricas.

In [ ]:
# Cargar datos de estudiantes (mismo CSV del notebook anterior)
CSV_PATH = "https://raw.githubusercontent.com/TesisEnel/Recopilacion_Datos_CalidadCodigo/refs/heads/main/Estudiantes_2023-2024.csv"

try:
    # Cargar el CSV con información de estudiantes
    df_estudiantes = pd.read_csv(CSV_PATH)
    
    print("📁 Datos de estudiantes cargados exitosamente")
    print(f"👥 Número de estudiantes: {len(df_estudiantes)}")
    print(f"📊 Columnas disponibles: {list(df_estudiantes.columns)}")
    
    # Verificar columnas de SonarCloud
    sonar_columns = [col for col in df_estudiantes.columns if 'Sonar' in col]
    print(f"\n🎯 Columnas de SonarCloud encontradas: {sonar_columns}")
    
    # Contar proyectos no vacíos
    sonar_ap1_count = df_estudiantes['Sonar_Ap1'].notna().sum()
    sonar_ap2_count = df_estudiantes['Sonar_Ap2'].notna().sum()
    
    print(f"📈 Proyectos Sonar_Ap1 disponibles: {sonar_ap1_count}")
    print(f"📈 Proyectos Sonar_Ap2 disponibles: {sonar_ap2_count}")
    print(f"📈 Total de proyectos para análisis de issues: {sonar_ap1_count + sonar_ap2_count}")
    
    # Mostrar muestra de datos
    print(f"\n🔍 Primeras 3 filas:")
    display(df_estudiantes[['ID', 'Estudiante', 'Sonar_Ap1', 'Sonar_Ap2']].head(3))
    
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo CSV")
    print(f"📍 Buscando en: {CSV_PATH}")
except Exception as e:
    print(f"❌ Error al cargar datos: {e}")

In [ ]:
# Extraer project keys de SonarCloud (reutilizando función del notebook anterior)
def extract_project_keys(df):
    """
    Extraer todas las claves de proyecto de SonarCloud del DataFrame
    """
    project_keys = []
    
    for index, row in df.iterrows():
        student_id = row.get('ID', f'Student_{index}')
        nombre = row.get('Estudiante', 'Unknown')
        
        # Procesar Sonar_Ap1
        if pd.notna(row['Sonar_Ap1']) and row['Sonar_Ap1'].strip():
            project_keys.append({
                'student_id': student_id,
                'nombre': nombre,
                'project_key': row['Sonar_Ap1'].strip(),
                'assignment': 'AP1',
                'row_index': index
            })
        
        # Procesar Sonar_Ap2
        if pd.notna(row['Sonar_Ap2']) and row['Sonar_Ap2'].strip():
            project_keys.append({
                'student_id': student_id,
                'nombre': nombre,
                'project_key': row['Sonar_Ap2'].strip(),
                'assignment': 'AP2',
                'row_index': index
            })
    
    return project_keys

# Extraer project keys
project_list = extract_project_keys(df_estudiantes)

print(f"🔑 Total de project keys extraídos: {len(project_list)}")

# Mostrar estadísticas por assignment
ap1_count = len([p for p in project_list if p['assignment'] == 'AP1'])
ap2_count = len([p for p in project_list if p['assignment'] == 'AP2'])

print(f"📊 Proyectos AP1: {ap1_count}")
print(f"📊 Proyectos AP2: {ap2_count}")

# Mostrar algunos ejemplos
print("\n🔍 Primeros 5 project keys para análisis de issues:")
for i, project in enumerate(project_list[:5]):
    print(f"  {i+1}. {project['nombre']} - {project['assignment']}: {project['project_key']}")

print(f"\n✅ Proyectos listos para extracción de issues")

## 3. Implementación de Funciones de Extracción de Issues

Implementamos las funciones principales para extraer issues detallados de SonarCloud utilizando el endpoint `/api/issues/search`.

### Funciones implementadas:

#### `fetch_project_issues(project_key)`
- **Propósito**: Obtener todos los issues de un proyecto específico desde SonarCloud
- **Endpoint**: `/api/issues/search`
- **Características**:
  - ⏱️ **Timing con decorador**: Mide tiempo de ejecución
  - 🔄 **Paginación automática**: Maneja múltiples páginas de resultados
  - 🛡️ **Manejo robusto de errores**: Captura errores HTTP, conexión y timeout
  - 📊 **Extracción completa**: Obtiene todos los campos relevantes del issue
  - 📝 **Logging estructurado**: Registra progreso y errores

#### `batch_fetch_issues(project_list)`
- **Propósito**: Procesar múltiples proyectos con control de rate limiting
- **Características**:
  - ⚡ **Procesamiento secuencial**: Evita sobrecargar la API
  - 📊 **Progreso en tiempo real**: Indicadores de avance por proyecto
  - 🎯 **Estadísticas detalladas**: Conteo de issues por proyecto
  - ⏳ **Rate limiting**: Delays configurables entre proyectos
  - 🛡️ **Tolerancia a fallos**: Continúa aunque fallen proyectos individuales

#### Funciones auxiliares:
- `parse_issue_data()`: Extrae y estructura datos del issue
- `create_issue_error_response()`: Respuestas de error estandarizadas

In [ ]:
# Funciones para extraer issues de SonarCloud API

def parse_issue_data(issue, project_info):
    """Extraer y estructurar datos de un issue individual"""
    return {
        # Información del proyecto y estudiante
        'student_id': project_info['student_id'],
        'nombre': project_info['nombre'],
        'assignment': project_info['assignment'],
        'row_index': project_info['row_index'],
        'project_key': project_info['project_key'],
        
        # Información del issue
        'issue_key': issue.get('key', ''),
        'rule': issue.get('rule', ''),
        'severity': issue.get('severity', ''),
        'type': issue.get('type', ''),
        'message': issue.get('message', ''),
        'component': issue.get('component', ''),
        'line': issue.get('line', 0),
        'status': issue.get('status', ''),
        'creation_date': issue.get('creationDate', ''),
        'update_date': issue.get('updateDate', ''),
        'effort': issue.get('effort', ''),
        'debt': issue.get('debt', ''),
        'tags': ','.join(issue.get('tags', [])) if issue.get('tags') else ''
    }

def create_issue_error_response(project_key, status, error_info=None):
    """Crear respuesta de error estandarizada para issues"""
    response = {
        'project_key': project_key, 
        'status': status,
        'issues_count': 0,
        'issues': []
    }
    if error_info:
        response['error'] = error_info
    return response

@timer_decorator
def fetch_project_issues(project_info: Dict, retries: int = 3, delay: float = 5.0) -> Dict:
    """
    Obtener todos los issues de un proyecto específico desde SonarCloud con paginación
    """
    project_key = project_info['project_key']
    all_issues = []
    page = 1
    has_more_pages = True
    
    while has_more_pages:
        url = f"{SONAR_BASE_URL}/issues/search"
        params = {
            'componentKeys': project_key,
            'p': page,
            'ps': ISSUES_CONFIG['page_size']
        }
        
        for attempt in range(retries):
            try:
                response = requests.get(
                    url, 
                    params=params, 
                    headers=get_auth_headers(), 
                    timeout=ISSUES_CONFIG['timeout']
                )
                
                if response.status_code == 200:
                    data = response.json()
                    
                    # Extraer issues de esta página
                    if 'issues' in data and data['issues']:
                        page_issues = [
                            parse_issue_data(issue, project_info) 
                            for issue in data['issues']
                        ]
                        all_issues.extend(page_issues)
                    
                    # Verificar si hay más páginas
                    paging = data.get('paging', {})
                    total = paging.get('total', 0)
                    page_size = paging.get('pageSize', ISSUES_CONFIG['page_size'])
                    
                    has_more_pages = (page * page_size) < total
                    if has_more_pages:
                        page += 1
                        print(f"    📄 Página {page-1} procesada, {len(page_issues)} issues encontrados")
                    else:
                        print(f"    📄 Página {page} procesada (final), {len(page_issues)} issues encontrados")
                    
                    break  # Salir del loop de reintentos
                    
                elif response.status_code == 401:
                    logger.warning(f"Authentication error for project {project_key}")
                    if attempt == retries - 1:
                        return create_issue_error_response(project_key, 'authentication_error', response.status_code)
                else:
                    logger.warning(f"HTTP error {response.status_code} for project {project_key}")
                    if attempt == retries - 1:
                        return create_issue_error_response(project_key, f'http_error_{response.status_code}', response.status_code)
                
                time.sleep(delay)
                
            except requests.exceptions.RequestException as e:
                logger.error(f"Connection error for project {project_key}: {e}")
                if attempt == retries - 1:
                    return create_issue_error_response(project_key, 'connection_error', str(e))
                time.sleep(delay)
            except Exception as e:
                logger.error(f"Unexpected error for project {project_key}: {e}")
                if attempt == retries - 1:
                    return create_issue_error_response(project_key, 'unexpected_error', str(e))
                time.sleep(delay)
    
    return {
        'project_key': project_key,
        'status': 'success',
        'issues_count': len(all_issues),
        'issues': all_issues
    }

@timer_decorator  
def batch_fetch_issues(project_list: List[Dict]) -> List[Dict]:
    """
    Obtener issues para múltiples proyectos con control de rate limiting
    """
    results = []
    total_projects = len(project_list)
    total_issues = 0
    
    print_section_header(f"🚀 Extracción de issues para {total_projects} proyectos")
    
    for i, project in enumerate(project_list, 1):
        project_key = project['project_key']
        print(f"\n📦 Proyecto {i}/{total_projects}: {project['nombre']} - {project['assignment']}")
        print(f"🔄 Procesando: {project_key}")
        
        # Extraer issues del proyecto
        result = fetch_project_issues(project)
        
        # Agregar información del estudiante al resultado
        result.update({
            'student_id': project['student_id'],
            'nombre': project['nombre'],
            'assignment': project['assignment'],
            'row_index': project['row_index']
        })
        
        results.append(result)
        
        # Mostrar resultado
        if result['status'] == 'success':
            issues_count = result['issues_count']
            total_issues += issues_count
            print(f"    ✅ {issues_count} issues extraídos exitosamente")
        else:
            print(f"    ❌ Error: {result.get('status', 'unknown')}")
        
        # Progress update
        progress = i / total_projects * 100
        print(f"    📊 Progreso: {progress:.1f}% ({i}/{total_projects})")
        
        # Delay entre proyectos para respetar rate limits
        if i < total_projects:
            print(f"    ⏳ Esperando {ISSUES_CONFIG['batch_delay']}s...")
            time.sleep(ISSUES_CONFIG['batch_delay'])
    
    # Estadísticas finales
    successful = [r for r in results if r.get('status') == 'success']
    print_section_header("📈 Estadísticas de Extracción de Issues")
    print(f"✅ Proyectos exitosos: {len(successful)} ({format_percentage(len(successful), total_projects)})")
    print(f"❌ Proyectos fallidos: {total_projects - len(successful)} ({format_percentage(total_projects - len(successful), total_projects)})")
    print(f"📊 Total de issues extraídos: {total_issues}")
    
    return results

print("🛠️  Funciones de extracción de issues configuradas")
print("📡 Listo para extraer issues detallados con paginación automática")

## 4. Ejecución del Proceso de Extracción de Issues

Ejecutamos el proceso de extracción de issues para todos los proyectos de estudiantes. Este proceso puede tomar considerablemente más tiempo que la extracción de métricas debido al volumen de datos por proyecto.

### Configuración del proceso:
- **Paginación**: 500 issues por página (máximo de SonarCloud)
- **Rate limiting**: 2.0 segundos entre proyectos
- **Reintentos**: 3 intentos por proyecto en caso de errores
- **Timeout**: 30 segundos por request

### Monitoreo incluido:
- ✅ **Progreso por proyecto**: Estudiante, assignment y project key
- 📄 **Progreso por página**: Número de issues por página procesada
- 📊 **Contadores en tiempo real**: Issues extraídos por proyecto
- ⏱️ **Tiempo de ejecución**: Por proyecto y total
- 🔍 **Detección de errores**: Logging de errores específicos

> **Nota**: La extracción puede tomar varios minutos dependiendo de la cantidad de issues por proyecto. Proyectos con muchos issues requerirán múltiples páginas.

In [ ]:
# Ejecutar extracción de issues
print("🚀 Iniciando proceso de extracción de issues detallados...")
print(f"📊 Total de proyectos a procesar: {len(project_list)}")
print(f"⚙️  Configuración: {ISSUES_CONFIG['page_size']} issues por página, {ISSUES_CONFIG['batch_delay']}s entre proyectos")

# Ejecutar extracción
issues_results = batch_fetch_issues(project_list)

print("\n📈 Resultados de la extracción de issues:")
print(f"✅ Total de proyectos procesados: {len(issues_results)}")

# Analizar resultados
successful_extractions = [r for r in issues_results if r.get('status') == 'success']
failed_extractions = [r for r in issues_results if r.get('status') != 'success']

print(f"✅ Extracciones exitosas: {len(successful_extractions)}")
print(f"❌ Extracciones fallidas: {len(failed_extractions)}")

# Calcular totales de issues
total_issues = sum(r.get('issues_count', 0) for r in successful_extractions)
print(f"📊 Total de issues extraídos: {total_issues}")

if failed_extractions:
    print("\n⚠️  Proyectos con errores:")
    for failed in failed_extractions[:5]:  # Mostrar solo los primeros 5
        print(f"  - {failed['project_key']}: {failed.get('status', 'unknown error')}")

# Mostrar estadísticas por assignment
ap1_projects = [r for r in successful_extractions if r.get('assignment') == 'AP1']
ap2_projects = [r for r in successful_extractions if r.get('assignment') == 'AP2']

ap1_issues = sum(r.get('issues_count', 0) for r in ap1_projects)
ap2_issues = sum(r.get('issues_count', 0) for r in ap2_projects)

print(f"\n📊 Distribución por assignment:")
print(f"  AP1: {len(ap1_projects)} proyectos, {ap1_issues} issues")
print(f"  AP2: {len(ap2_projects)} proyectos, {ap2_issues} issues")

# Mostrar ejemplo de issues extraídos
if successful_extractions and total_issues > 0:
    # Buscar el primer proyecto con issues
    example_project = None
    for project in successful_extractions:
        if project.get('issues_count', 0) > 0:
            example_project = project
            break
    
    if example_project:
        print(f"\n🔍 Ejemplo de issues extraídos (proyecto: {example_project['project_key']}):")
        print(f"Estudiante: {example_project['nombre']}")
        print(f"Assignment: {example_project['assignment']}")
        print(f"Total de issues: {example_project['issues_count']}")
        
        # Mostrar primeros 3 issues
        sample_issues = example_project['issues'][:3]
        for i, issue in enumerate(sample_issues, 1):
            print(f"\n  Issue {i}:")
            print(f"    Tipo: {issue['type']}")
            print(f"    Severidad: {issue['severity']}")
            print(f"    Regla: {issue['rule']}")
            print(f"    Línea: {issue['line']}")
            print(f"    Mensaje: {issue['message'][:100]}{'...' if len(issue['message']) > 100 else ''}")

print("\n✅ Extracción de issues completada")

## 5. Procesamiento y Estructuración de Datos de Issues

Convertimos los resultados de la extracción en DataFrames estructurados para facilitar el análisis. A diferencia de las métricas, los issues requieren un procesamiento más complejo debido a su naturaleza detallada.

### Procesamiento realizado:
1. **Aplanamiento de datos**: Convertir estructura anidada en DataFrame plano
2. **Limpieza de mensajes**: Preparar mensajes para formato CSV
3. **Categorización**: Agrupar por tipo, severidad y regla
4. **Validación**: Verificar integridad y completitud de datos

### DataFrames generados:
- **`df_all_issues`**: Todos los issues individuales con información completa
- **`df_issues_summary`**: Resumen agregado por proyecto
- **`df_issues_by_type`**: Análisis por tipo de issue (BUG, VULNERABILITY, CODE_SMELL)
- **`df_issues_by_severity`**: Análisis por severidad (CRITICAL, MAJOR, MINOR, etc.)

### Variables de análisis:
- Total de issues por estudiante y assignment
- Distribución por categorías de calidad
- Análisis de reglas más frecuentes
- Patrones de ubicación de problemas

In [ ]:
# Procesar y estructurar los datos extraídos de issues
print("🔄 Procesando datos de issues extraídos...")

# 1. Crear DataFrame con todos los issues individuales
all_issues_data = []

for project_result in successful_extractions:
    if project_result.get('issues'):
        all_issues_data.extend(project_result['issues'])

if all_issues_data:
    df_all_issues = pd.DataFrame(all_issues_data)
    print(f"📊 DataFrame de issues creado con {len(df_all_issues)} issues individuales")
    
    # Mostrar información básica del DataFrame
    print(f"📋 Columnas disponibles: {len(df_all_issues.columns)}")
    print(f"👥 Estudiantes únicos: {df_all_issues['student_id'].nunique()}")
    print(f"🎯 Proyectos únicos: {df_all_issues['project_key'].nunique()}")
    
    # 2. Crear resumen por proyecto
    df_issues_summary = df_all_issues.groupby([
        'student_id', 'nombre', 'project_key', 'assignment'
    ]).agg({
        'issue_key': 'count',
        'type': lambda x: x.value_counts().to_dict(),
        'severity': lambda x: x.value_counts().to_dict()
    }).reset_index()
    
    df_issues_summary.columns = ['student_id', 'nombre', 'project_key', 'assignment', 
                                'total_issues', 'types_distribution', 'severity_distribution']
    
    print(f"📈 Resumen por proyecto creado con {len(df_issues_summary)} proyectos")
    
    # 3. Análisis por tipo de issue
    type_analysis = df_all_issues.groupby(['assignment', 'type']).size().reset_index(name='count')
    type_pivot = type_analysis.pivot(index='assignment', columns='type', values='count').fillna(0)
    
    print(f"\n📊 Distribución por tipo de issue:")
    for assignment in ['AP1', 'AP2']:
        if assignment in type_pivot.index:
            print(f"  {assignment}:")
            for issue_type in type_pivot.columns:
                count = int(type_pivot.loc[assignment, issue_type])
                if count > 0:
                    print(f"    {issue_type}: {count}")
    
    # 4. Análisis por severidad
    severity_analysis = df_all_issues.groupby(['assignment', 'severity']).size().reset_index(name='count')
    severity_pivot = severity_analysis.pivot(index='assignment', columns='severity', values='count').fillna(0)
    
    print(f"\n🚨 Distribución por severidad:")
    for assignment in ['AP1', 'AP2']:
        if assignment in severity_pivot.index:
            print(f"  {assignment}:")
            for severity in severity_pivot.columns:
                count = int(severity_pivot.loc[assignment, severity])
                if count > 0:
                    print(f"    {severity}: {count}")
    
    # 5. Top 10 reglas más frecuentes
    top_rules = df_all_issues['rule'].value_counts().head(10)
    print(f"\n📋 Top 10 reglas más frecuentes:")
    for i, (rule, count) in enumerate(top_rules.items(), 1):
        print(f"  {i}. {rule}: {count} issues")
    
    # 6. Análisis de estudiantes con más issues
    student_issues = df_all_issues.groupby(['student_id', 'nombre', 'assignment']).size().reset_index(name='issues_count')
    top_students = student_issues.nlargest(10, 'issues_count')
    
    print(f"\n👥 Top 10 estudiantes con más issues:")
    for _, student in top_students.iterrows():
        print(f"  {student['nombre']} ({student['assignment']}): {student['issues_count']} issues")
    
    # 7. Mostrar muestra del DataFrame principal
    print(f"\n🔍 Primeras 3 filas del dataset de issues:")
    display(df_all_issues[['nombre', 'assignment', 'project_key', 'type', 'severity', 'rule', 'line', 'message']].head(3))
    
    # 8. Crear DataFrames específicos para análisis
    df_issues_by_type = df_all_issues.groupby(['student_id', 'nombre', 'assignment', 'type']).size().reset_index(name='count')
    df_issues_by_severity = df_all_issues.groupby(['student_id', 'nombre', 'assignment', 'severity']).size().reset_index(name='count')
    
    print(f"\n✅ DataFrames de análisis creados:")
    print(f"  📊 Issues por tipo: {len(df_issues_by_type)} filas")
    print(f"  🚨 Issues por severidad: {len(df_issues_by_severity)} filas")

else:
    print("❌ No se encontraron issues para procesar")
    df_all_issues = pd.DataFrame()
    df_issues_summary = pd.DataFrame()

print("✅ Procesamiento de datos de issues completado")

## 6. Análisis Detallado por Categorías

Esta sección proporciona análisis profundos de los issues extraídos organizados por diferentes categorías:

1. **Análisis por Tipo de Issue**: Distribución de BUG, CODE_SMELL, VULNERABILITY, SECURITY_HOTSPOT
2. **Análisis por Severidad**: Distribución de CRITICAL, MAJOR, MINOR, INFO
3. **Análisis por Regla**: Identificación de las reglas más violadas
4. **Análisis Temporal**: Comparación entre asignaciones (AP1 vs AP2)
5. **Análisis Estadístico**: Métricas descriptivas y correlaciones

In [ ]:
# Análisis detallado por categorías
if not df_all_issues.empty:
    print("📊 Iniciando análisis detallado por categorías...")
    
    # 1. ANÁLISIS POR TIPO DE ISSUE
    print("\n" + "="*60)
    print("📋 ANÁLISIS POR TIPO DE ISSUE")
    print("="*60)
    
    # Distribución general por tipo
    type_counts = df_all_issues['type'].value_counts()
    total_issues = len(df_all_issues)
    
    print(f"📊 Distribución de tipos (Total: {total_issues} issues):")
    for issue_type, count in type_counts.items():
        percentage = (count / total_issues) * 100
        print(f"  {issue_type}: {count} ({percentage:.1f}%)")
    
    # Análisis por asignación
    type_by_assignment = df_all_issues.groupby(['assignment', 'type']).size().unstack(fill_value=0)
    print(f"\n📈 Distribución por asignación:")
    for assignment in type_by_assignment.index:
        total_assignment = type_by_assignment.loc[assignment].sum()
        print(f"  {assignment} (Total: {total_assignment}):")
        for issue_type in type_by_assignment.columns:
            count = type_by_assignment.loc[assignment, issue_type]
            if count > 0:
                percentage = (count / total_assignment) * 100
                print(f"    {issue_type}: {count} ({percentage:.1f}%)")
    
    # 2. ANÁLISIS POR SEVERIDAD
    print("\n" + "="*60)
    print("🚨 ANÁLISIS POR SEVERIDAD")
    print("="*60)
    
    # Distribución general por severidad
    severity_counts = df_all_issues['severity'].value_counts()
    print(f"🚨 Distribución de severidades:")
    for severity, count in severity_counts.items():
        percentage = (count / total_issues) * 100
        print(f"  {severity}: {count} ({percentage:.1f}%)")
    
    # Análisis por asignación
    severity_by_assignment = df_all_issues.groupby(['assignment', 'severity']).size().unstack(fill_value=0)
    print(f"\n📈 Distribución por asignación:")
    for assignment in severity_by_assignment.index:
        total_assignment = severity_by_assignment.loc[assignment].sum()
        print(f"  {assignment} (Total: {total_assignment}):")
        for severity in severity_by_assignment.columns:
            count = severity_by_assignment.loc[assignment, severity]
            if count > 0:
                percentage = (count / total_assignment) * 100
                print(f"    {severity}: {count} ({percentage:.1f}%)")
    
    # 3. ANÁLISIS POR REGLAS MÁS FRECUENTES
    print("\n" + "="*60)
    print("📋 ANÁLISIS POR REGLAS")
    print("="*60)
    
    # Top 15 reglas más frecuentes
    top_rules = df_all_issues['rule'].value_counts().head(15)
    print(f"📋 Top 15 reglas más violadas:")
    for i, (rule, count) in enumerate(top_rules.items(), 1):
        percentage = (count / total_issues) * 100
        print(f"  {i:2d}. {rule}: {count} ({percentage:.1f}%)")
    
    # Análisis de reglas por tipo de issue
    print(f"\n🔍 Distribución de reglas por tipo:")
    rules_by_type = df_all_issues.groupby(['type', 'rule']).size().reset_index(name='count')
    for issue_type in df_all_issues['type'].unique():
        top_rules_type = rules_by_type[rules_by_type['type'] == issue_type].nlargest(5, 'count')
        print(f"  {issue_type}:")
        for _, row in top_rules_type.iterrows():
            print(f"    {row['rule']}: {row['count']}")
    
    # 4. ANÁLISIS ESTADÍSTICO DESCRIPTIVO
    print("\n" + "="*60)
    print("📊 ANÁLISIS ESTADÍSTICO")
    print("="*60)
    
    # Issues por estudiante
    issues_per_student = df_all_issues.groupby(['student_id', 'nombre', 'assignment']).size().reset_index(name='issues_count')
    
    print(f"👥 Estadísticas de issues por estudiante:")
    for assignment in ['AP1', 'AP2']:
        assignment_data = issues_per_student[issues_per_student['assignment'] == assignment]['issues_count']
        if not assignment_data.empty:
            print(f"  {assignment}:")
            print(f"    Promedio: {assignment_data.mean():.1f}")
            print(f"    Mediana: {assignment_data.median():.1f}")
            print(f"    Desv. Estándar: {assignment_data.std():.1f}")
            print(f"    Mínimo: {assignment_data.min()}")
            print(f"    Máximo: {assignment_data.max()}")
    
    # Issues por proyecto
    issues_per_project = df_all_issues.groupby(['project_key', 'assignment']).size().reset_index(name='issues_count')
    
    print(f"\n📁 Estadísticas de issues por proyecto:")
    for assignment in ['AP1', 'AP2']:
        assignment_data = issues_per_project[issues_per_project['assignment'] == assignment]['issues_count']
        if not assignment_data.empty:
            print(f"  {assignment}:")
            print(f"    Promedio: {assignment_data.mean():.1f}")
            print(f"    Mediana: {assignment_data.median():.1f}")
            print(f"    Proyectos totales: {len(assignment_data)}")
    
    # 5. COMPARACIÓN ENTRE ASIGNACIONES
    print("\n" + "="*60)
    print("🔄 COMPARACIÓN AP1 vs AP2")
    print("="*60)
    
    ap1_issues = df_all_issues[df_all_issues['assignment'] == 'AP1']
    ap2_issues = df_all_issues[df_all_issues['assignment'] == 'AP2']
    
    print(f"📊 Resumen comparativo:")
    print(f"  AP1: {len(ap1_issues)} issues en {ap1_issues['project_key'].nunique()} proyectos")
    print(f"  AP2: {len(ap2_issues)} issues en {ap2_issues['project_key'].nunique()} proyectos")
    
    if len(ap1_issues) > 0 and len(ap2_issues) > 0:
        print(f"  Promedio issues/proyecto:")
        print(f"    AP1: {len(ap1_issues) / ap1_issues['project_key'].nunique():.1f}")
        print(f"    AP2: {len(ap2_issues) / ap2_issues['project_key'].nunique():.1f}")
    
    print("\n✅ Análisis detallado completado")
    
else:
    print("❌ No hay datos de issues disponibles para el análisis")

## 7. Exportar Datos de Issues

Esta sección se encarga de exportar los datos extraídos y procesados a archivos CSV para su uso posterior:

1. **Issues Individuales**: Archivo con todos los issues detallados
2. **Resumen por Proyecto**: Consolidado con conteos y distribuciones
3. **Análisis por Tipo**: Distribución de tipos de issues por estudiante/proyecto
4. **Análisis por Severidad**: Distribución de severidades por estudiante/proyecto
5. **Metadatos de Extracción**: Información sobre el proceso de extracción

In [ ]:
# Exportar datos de issues a archivos CSV
import os
from datetime import datetime

# Crear directorio data si no existe
os.makedirs('../data', exist_ok=True)

# Timestamp para archivos
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

if not df_all_issues.empty:
    print("📁 Exportando datos de issues...")
    
    try:
        # 1. EXPORTAR ISSUES INDIVIDUALES COMPLETOS
        issues_filename = f'../data/issues_detallados_{timestamp}.csv'
        df_all_issues.to_csv(issues_filename, index=False, encoding='utf-8-sig')
        print(f"✅ Issues individuales exportados: {issues_filename}")
        print(f"   📊 {len(df_all_issues)} issues de {df_all_issues['student_id'].nunique()} estudiantes")
        
        # 2. EXPORTAR RESUMEN POR PROYECTO
        if not df_issues_summary.empty:
            summary_filename = f'../data/issues_resumen_proyecto_{timestamp}.csv'
            df_issues_summary.to_csv(summary_filename, index=False, encoding='utf-8-sig')
            print(f"✅ Resumen por proyecto exportado: {summary_filename}")
            print(f"   📊 {len(df_issues_summary)} proyectos")
        
        # 3. EXPORTAR ANÁLISIS POR TIPO
        if 'df_issues_by_type' in locals():
            type_filename = f'../data/issues_por_tipo_{timestamp}.csv'
            df_issues_by_type.to_csv(type_filename, index=False, encoding='utf-8-sig')
            print(f"✅ Análisis por tipo exportado: {type_filename}")
            print(f"   📊 {len(df_issues_by_type)} registros")
        
        # 4. EXPORTAR ANÁLISIS POR SEVERIDAD
        if 'df_issues_by_severity' in locals():
            severity_filename = f'../data/issues_por_severidad_{timestamp}.csv'
            df_issues_by_severity.to_csv(severity_filename, index=False, encoding='utf-8-sig')
            print(f"✅ Análisis por severidad exportado: {severity_filename}")
            print(f"   📊 {len(df_issues_by_severity)} registros")
        
        # 5. CREAR ARCHIVO DE ESTADÍSTICAS RESUMIDAS
        stats_data = []
        
        # Estadísticas generales
        stats_data.append({
            'categoria': 'General',
            'metrica': 'Total Issues',
            'valor': len(df_all_issues),
            'descripcion': 'Número total de issues extraídos'
        })
        
        stats_data.append({
            'categoria': 'General',
            'metrica': 'Estudiantes Únicos',
            'valor': df_all_issues['student_id'].nunique(),
            'descripcion': 'Número de estudiantes con issues'
        })
        
        stats_data.append({
            'categoria': 'General',
            'metrica': 'Proyectos Únicos',
            'valor': df_all_issues['project_key'].nunique(),
            'descripcion': 'Número de proyectos con issues'
        })
        
        # Estadísticas por asignación
        for assignment in ['AP1', 'AP2']:
            assignment_issues = df_all_issues[df_all_issues['assignment'] == assignment]
            if not assignment_issues.empty:
                stats_data.append({
                    'categoria': assignment,
                    'metrica': 'Total Issues',
                    'valor': len(assignment_issues),
                    'descripcion': f'Issues en {assignment}'
                })
                
                stats_data.append({
                    'categoria': assignment,
                    'metrica': 'Proyectos',
                    'valor': assignment_issues['project_key'].nunique(),
                    'descripcion': f'Proyectos con issues en {assignment}'
                })
                
                stats_data.append({
                    'categoria': assignment,
                    'metrica': 'Promedio Issues/Proyecto',
                    'valor': round(len(assignment_issues) / assignment_issues['project_key'].nunique(), 2),
                    'descripcion': f'Promedio de issues por proyecto en {assignment}'
                })
        
        # Estadísticas por tipo de issue
        type_counts = df_all_issues['type'].value_counts()
        for issue_type, count in type_counts.items():
            stats_data.append({
                'categoria': 'Tipos',
                'metrica': issue_type,
                'valor': count,
                'descripcion': f'Issues de tipo {issue_type}'
            })
        
        # Estadísticas por severidad
        severity_counts = df_all_issues['severity'].value_counts()
        for severity, count in severity_counts.items():
            stats_data.append({
                'categoria': 'Severidad',
                'metrica': severity,
                'valor': count,
                'descripcion': f'Issues con severidad {severity}'
            })
        
        # Exportar estadísticas
        df_stats = pd.DataFrame(stats_data)
        stats_filename = f'../data/issues_estadisticas_{timestamp}.csv'
        df_stats.to_csv(stats_filename, index=False, encoding='utf-8-sig')
        print(f"✅ Estadísticas exportadas: {stats_filename}")
        print(f"   📊 {len(df_stats)} métricas estadísticas")
        
        # 6. CREAR ARCHIVO DE METADATOS
        metadata = {
            'fecha_extraccion': datetime.now().isoformat(),
            'total_estudiantes_procesados': len(df_estudiantes) if 'df_estudiantes' in locals() else 0,
            'total_proyectos_intentados': len(successful_extractions) + len(failed_extractions) if 'successful_extractions' in locals() and 'failed_extractions' in locals() else 0,
            'total_proyectos_exitosos': len(successful_extractions) if 'successful_extractions' in locals() else 0,
            'total_proyectos_fallidos': len(failed_extractions) if 'failed_extractions' in locals() else 0,
            'total_issues_extraidos': len(df_all_issues),
            'estudiantes_con_issues': df_all_issues['student_id'].nunique(),
            'proyectos_con_issues': df_all_issues['project_key'].nunique(),
            'archivos_generados': [
                issues_filename,
                summary_filename if not df_issues_summary.empty else None,
                type_filename if 'df_issues_by_type' in locals() else None,
                severity_filename if 'df_issues_by_severity' in locals() else None,
                stats_filename
            ]
        }
        
        # Filtrar archivos None
        metadata['archivos_generados'] = [f for f in metadata['archivos_generados'] if f is not None]
        
        metadata_filename = f'../data/issues_metadata_{timestamp}.json'
        import json
        with open(metadata_filename, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Metadatos exportados: {metadata_filename}")
        
        # RESUMEN FINAL
        print(f"\n" + "="*60)
        print("📋 RESUMEN DE EXPORTACIÓN")
        print("="*60)
        print(f"🕒 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 Issues exportados: {len(df_all_issues)}")
        print(f"👥 Estudiantes: {df_all_issues['student_id'].nunique()}")
        print(f"📁 Proyectos: {df_all_issues['project_key'].nunique()}")
        print(f"📄 Archivos generados: {len(metadata['archivos_generados'])}")
        print(f"\n📁 Archivos guardados en: ../data/")
        for archivo in metadata['archivos_generados']:
            print(f"   • {os.path.basename(archivo)}")
        
        # También crear una versión simplificada sin timestamp para facilitar el acceso
        main_filename = '../data/issues_detallados_latest.csv'
        df_all_issues.to_csv(main_filename, index=False, encoding='utf-8-sig')
        print(f"\n✅ Archivo principal actualizado: {main_filename}")
        
    except Exception as e:
        print(f"❌ Error durante la exportación: {str(e)}")
        logging.error(f"Error en exportación: {str(e)}")

else:
    print("❌ No hay datos de issues para exportar")

print("\n🎉 Proceso de extracción de issues completado!")

## 📋 Resumen del Notebook

Este notebook ha completado exitosamente la extracción y análisis de issues de SonarCloud para los proyectos de estudiantes. 

### ✅ Procesos Realizados:

1. **Configuración**: Setup de APIs y credenciales de SonarCloud
2. **Carga de Datos**: Lectura de información de estudiantes y proyectos
3. **Extracción**: Obtención de issues detallados usando la API de SonarCloud
4. **Procesamiento**: Estructuración y análisis de los datos extraídos
5. **Análisis**: Categorización por tipo, severidad, reglas y comparaciones
6. **Exportación**: Generación de múltiples archivos CSV para análisis posterior

### 📊 Datos Generados:

- **Issues Detallados**: Cada issue individual con toda su información
- **Resumen por Proyecto**: Consolidación de issues por proyecto y estudiante  
- **Análisis por Categorías**: Distribuciones por tipo y severidad
- **Estadísticas**: Métricas descriptivas y comparativas
- **Metadatos**: Información del proceso de extracción

### 🔄 Próximos Pasos:

1. Utilizar los archivos CSV generados para análisis estadísticos avanzados
2. Combinar con datos de métricas de calidad para análisis integral
3. Generar visualizaciones y reportes de calidad de software
4. Identificar patrones y tendencias en los tipos de issues más comunes

---
*Notebook creado para el análisis de calidad de software en proyectos estudiantiles - Tesis Aplicada 2*